<a href="https://colab.research.google.com/github/flahbocchino/farmtech-fase7-grupoFlavia-/blob/main/fase3_iot_esp32/FarmTech_Fase3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd
import random
from datetime import datetime


In [2]:
conn = sqlite3.connect("farmtech.db")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()


[('talhoes',), ('leituras_sensores',), ('sqlite_sequence',)]

In [3]:
def simular_leitura_sensores(talhao_id: int) -> dict:
    return {
        "talhao_id": talhao_id,
        "data_hora": datetime.now().isoformat(sep=" ", timespec="seconds"),
        "umidade": round(random.uniform(10, 80), 1),
        "ph": round(random.uniform(4.5, 7.5), 1),
        "nutrientes": round(random.uniform(20, 100), 1),
        "origem": "simulado"
    }


In [4]:
def inserir_leitura_no_banco(conn, leitura: dict):
    cursor = conn.cursor()
    cursor.execute(
        """
        INSERT INTO leituras_sensores
        (talhao_id, data_hora, umidade, ph, nutrientes, origem)
        VALUES (?, ?, ?, ?, ?, ?)
        """,
        (
            leitura["talhao_id"],
            leitura["data_hora"],
            leitura["umidade"],
            leitura["ph"],
            leitura["nutrientes"],
            leitura["origem"],
        )
    )
    conn.commit()


In [5]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS leituras_sensores (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    talhao_id INTEGER,
    data_hora TEXT,
    umidade REAL,
    ph REAL,
    nutrientes REAL,
    origem TEXT,
    FOREIGN KEY (talhao_id) REFERENCES talhoes(talhao_id)
);
""")
conn.commit()


In [6]:
def avaliar_irrigacao(leitura: dict) -> str:
    if leitura["umidade"] < 30:
        return "LIGAR BOMBA por 15 minutos"
    else:
        return "NÃO ligar bomba (umidade suficiente)"


In [7]:
def rodar_ciclo_leitura(conn, talhao_id: int = 1):
    leitura = simular_leitura_sensores(talhao_id)
    acao = avaliar_irrigacao(leitura)
    inserir_leitura_no_banco(conn, leitura)
    return leitura, acao

# Teste
for i in range(3):
    leitura, acao = rodar_ciclo_leitura(conn, talhao_id=1)
    print(leitura, "->", acao)


{'talhao_id': 1, 'data_hora': '2025-11-22 23:38:44', 'umidade': 58.9, 'ph': 5.1, 'nutrientes': 67.3, 'origem': 'simulado'} -> NÃO ligar bomba (umidade suficiente)
{'talhao_id': 1, 'data_hora': '2025-11-22 23:38:44', 'umidade': 73.5, 'ph': 6.7, 'nutrientes': 29.7, 'origem': 'simulado'} -> NÃO ligar bomba (umidade suficiente)
{'talhao_id': 1, 'data_hora': '2025-11-22 23:38:44', 'umidade': 73.7, 'ph': 4.6, 'nutrientes': 81.4, 'origem': 'simulado'} -> NÃO ligar bomba (umidade suficiente)


In [8]:
df_leituras = pd.read_sql_query(
    "SELECT * FROM leituras_sensores ORDER BY id DESC LIMIT 10;",
    conn
)
df_leituras


,id,talhao_id,data_hora,umidade,ph,nutrientes,origem
0,3,1,2025-11-22 23:38:44,73.7,4.6,81.4,simulado
1,2,1,2025-11-22 23:38:44,73.5,6.7,29.7,simulado
2,1,1,2025-11-22 23:38:44,58.9,5.1,67.3,simulado


In [9]:
conn.close()
